<a href="https://colab.research.google.com/github/hugosousa111/segmentacao_base_ph2/blob/main/pr%C3%A9_processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurações e Bibliotecas

In [1]:
# Acessa o Drive
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt 
import numpy as np 
from scipy import signal as sg
import cv2 as cv
import warnings
warnings.filterwarnings("ignore") 
import os
from IPython.display import clear_output
import pandas as pd
from collections import deque 

# Funções Auxiliares

In [3]:
def just_save(image, caminho="", nome="name",  colorida=False):
    dpi = 80

    if not colorida:
        height, width = image.shape

        figsize = width / float(dpi), height / float(dpi)

        fig = plt.figure(figsize=figsize)
        ax = fig.add_axes([0, 0, 1, 1])
        ax.axis('off')

        ax.imshow(image, cmap=plt.cm.gray)
    else:
        height, width, nbands = image.shape

        figsize = width / float(dpi), height / float(dpi)

        fig = plt.figure(figsize=figsize)
        ax = fig.add_axes([0, 0, 1, 1])

        ax.axis('off')

        ax.imshow(image[:,:,::-1])
    ax.set(xlim=[-0.5, width - 0.5], ylim=[height - 0.5, -0.5], aspect=1)

    fig.savefig('/content/drive/My Drive/PH2_TCC/'+caminho+'/'+nome, dpi=dpi, transparent=True)
    plt.close(fig)

In [4]:
def get_names_images():
    filenames= os.listdir ("/content/drive/My Drive/PH2Dataset/PH2 Dataset images")

    result = []
    for filename in filenames:
        if os.path.isdir(os.path.join(os.path.abspath("/content/drive/My Drive/PH2Dataset/PH2 Dataset images"), filename)):
            result.append(filename)        
    result.sort()
    return result

imagens = get_names_images()
#imagens = imagens[0:30]

In [5]:
def carrega_imagem_original(image, color=0):
    img = cv.imread('/content/drive/My Drive/PH2Dataset/PH2 Dataset images/'+image+'/'+image+'_Dermoscopic_Image/'+image+'.bmp', color)
    return img

# Pré Processamento

In [6]:
def pre_processamento_blackhat(src, plot=0):
    grayScale = cv.cvtColor( src, cv.COLOR_RGB2GRAY )

    kernel = cv.getStructuringElement(1,(15,15))
    blackhat = cv.morphologyEx(grayScale, cv.MORPH_BLACKHAT, kernel)

    ret,thresh2 = cv.threshold(blackhat,10,255,cv.THRESH_BINARY)
    
    dst = cv.inpaint(src,thresh2,1,cv.INPAINT_TELEA)
    if plot==1: 
        plota(grayScale, "")
        just_save(grayScale, caminho="outros", nome="grayScale",  colorida=False)
    
        plota(blackhat, "")
        just_save(blackhat, caminho="outros", nome="blackhat",  colorida=False)

        plota(thresh2, "")
        just_save(thresh2, caminho="outros", nome="thresh2",  colorida=False)
    return dst

In [7]:
def pre_processamento_array(imagens, tipo=0): 
    if tipo == 0:
        dir = "Pre_Processadas_Black_Hat"
        for img_nome in imagens:
            print(img_nome+' ...')
            img = carrega_imagem_original(img_nome, 1)

            img_pre = pre_processamento_blackhat(img)

            just_save(img_pre,dir,img_nome, True)
    clear_output()

In [8]:
# Pré-processamento
imagens = get_names_images()
pre_processamento_array(imagens)